In [12]:
%matplotlib inline

import requests
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt


In [2]:
def get_data_from_url(base_url, index):
    response = requests.get(base_url)
    data = response.json()
    headers = data['resultSets'][index]['headers']
    rows = data['resultSets'][index]['rowSet']
    shot_df = pd.DataFrame(rows, columns=headers)
    return shot_df


In [14]:
stats = get_data_from_url("http://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=1996-97&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=", 0)
seasons = ["1997-98","1998-99","1999-00","2000-01","2001-02","2002-03","2003-04"
          ,"2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12"
          ,"2012-13","2013-14","2014-15","2015-16"]
for s in seasons:
    url = """http://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=""" % s
    stats = stats.append(get_data_from_url(url, 0))

print stats.head()

      TEAM_ID            TEAM_NAME  GP   W   L  W_PCT   MIN  OFF_RATING  \
0  1610612737        Atlanta Hawks  82  56  26  0.683  4076       105.4   
1  1610612738       Boston Celtics  82  15  67  0.183  4111       100.8   
2  1610612766    Charlotte Hornets  82  54  28  0.659  4086       108.0   
3  1610612741        Chicago Bulls  82  69  13  0.841  4069       111.1   
4  1610612739  Cleveland Cavaliers  82  42  40  0.512  4115       102.0   

   DEF_RATING  NET_RATING         ...           OREB_PCT  DREB_PCT  REB_PCT  \
0        98.9         6.5         ...              0.315     0.692    0.508   
1       108.8        -8.0         ...              0.294     0.685    0.475   
2       105.6         2.4         ...              0.287     0.681    0.490   
3        99.2        12.0         ...              0.359     0.693    0.529   
4        99.4         2.6         ...              0.294     0.719    0.503   

   TM_TOV_PCT  EFG_PCT  TS_PCT   PACE    PIE  CFID             CFPARAMS  


In [18]:
data = get_data_from_url('http://stats.nba.com/stats/leaguegamelog?Counter=1000&Direction=DESC&LeagueID=00&PlayerOrTeam=T&Season=2015-16&SeasonType=Regular+Season&Sorter=PTS', 0)


In [32]:
data.sort(['GAME_ID'], ascending=[False])
wins = data[data['WL']=='W']
losses = data[data['WL']=='L']

games = pd.merge(wins, losses, on='GAME_ID', how='outer')
games = games.sort(['GAME_ID'], ascending=[False])

print games.head()


    SEASON_ID_x   TEAM_ID_x TEAM_ABBREVIATION_x             TEAM_NAME_x  \
18        22015  1610612744                 GSW   Golden State Warriors   
95        22015  1610612757                 POR  Portland Trail Blazers   
165       22015  1610612762                 UTA               Utah Jazz   
96        22015  1610612759                 SAS       San Antonio Spurs   
224       22015  1610612742                 DAL        Dallas Mavericks   

        GAME_ID GAME_DATE_x    MATCHUP_x WL_x  MIN_x  FGM_x  \
18   0021500245  2015-11-28  GSW vs. SAC    W    240     39   
95   0021500244  2015-11-28  POR vs. LAL    W    240     38   
165  0021500243  2015-11-28  UTA vs. NOP    W    240     38   
96   0021500242  2015-11-28  SAS vs. ATL    W    240     40   
224  0021500241  2015-11-28  DAL vs. DEN    W    240     33   

           ...          DREB_y  REB_y  AST_y  STL_y  BLK_y  TOV_y  PF_y  \
18         ...              29     48     22      9      7     15    24   
95         ...      

/home/scott/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/home/scott/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [16]:
stats.sort(['W_PCT'], ascending=[False])

Y= stats.W_PCT
X = stats[['TS_PCT', 'DREB_PCT', 'OREB_PCT', 'TM_TOV_PCT','DEF_RATING']]
X = sm.add_constant(X)

model = sm.OLS(Y,X)
results = model.fit()
results.summary()

/home/scott/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  W_PCT   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     1781.
Date:                Sun, 29 Nov 2015   Prob (F-statistic):               0.00
Time:                        07:42:38   Log-Likelihood:                 1075.1
No. Observations:                 592   AIC:                            -2138.
Df Residuals:                     586   BIC:                            -2112.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.9864      0.106      9.268      0.000         0.777     1.195
TS_PCT         5.0289      0.087     57.905      0.000         4.858     5.199
DREB_PCT       0.0405      0.076      0.535      0.593        -0.108     0.189
OREB_PCT       1.5744      0.066     23.802      0.000         1.445     1.704
TM_TOV_PCT    -3.8567      0.147    -26.307      0.000        -4.145    -3.569
DEF_RATING    -0.0294      0.000    -63.754      0.000        -0.030    -0.029
==============================================================================
Omnibus:                        8.311   Durbin-Watson:                   2.133
Prob(Omnibus):                  0.016   Jarque-Bera (JB):                9.235
Skew:                           0.204   Prob(JB):                      0.00988
Kurtosis:                       3.456   Cond. No.                     1.01e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""